In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

data = pd.read_csv("marketing_customer_analysis.csv")

data.describe()

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
data.isna().sum()

In [ ]:
data["Response"].unique()

In [ ]:
num_data = data.select_dtypes (include = "number")

In [ ]:
cols = num_data.columns

for col in cols:
    sns.boxplot(num_data[col])
    print ("boxplot for numerical data: ",col)
    plt.show()
    sns.distplot(num_data[col])
    print ("distplot for numerical data: ", col)
    plt.show()
    
outliers = ["Customer Lifetime Value", "Monthly Premium Auto", "Number of Policies", "Total Claim Amount"]
cleaned = data

for out in outliers:
    IQR1 = data[out].quantile(0.25) - 1.5*(data[out].quantile(0.75) - data[out].quantile(0.25))
    IQR2 = data[out].quantile(0.75) + 1.5*(data[out].quantile(0.75) - data[out].quantile(0.25))
    cleaned = cleaned[(cleaned[out] > IQR1) & (cleaned[out] < IQR2)]
    
cleaned

In [ ]:
##########
num_cleaned = cleaned.select_dtypes (include = "number")
str_cleaned = cleaned.select_dtypes (include = "object")

coll = num_cleaned.columns

for col in cols:
    sns.boxplot(num_cleaned[col])
    print ("boxplot for numerical data: ",col)
    plt.show()
###########

In [ ]:
num_cleaned = cleaned.select_dtypes (include = "number")
str_cleaned = cleaned.select_dtypes (include = "object")

num_cleaned

In [ ]:
str_cleaned

In [ ]:
y = num_cleaned["Total Claim Amount"]
x = num_cleaned.drop("Total Claim Amount", axis = 1)

from sklearn.preprocessing import Normalizer
transformer = Normalizer().fit(x)
x_normalized = transformer.transform(x)

In [ ]:
df_norm = pd.DataFrame(x_normalized)

In [ ]:
df_norm

In [ ]:
str_cleaned

In [ ]:
str_cleaned.reset_index(inplace = True)

In [ ]:
str_cleaned.drop("index", inplace = True, axis = 1)

In [ ]:
str_cleaned.drop("Customer", inplace = True, axis = 1)

In [ ]:
str_cleaned.columns

In [ ]:
for col in str_cleaned.columns:
    print("Dummification of the col: ",col)
    print(pd.get_dummies(str_cleaned[col], drop_first = True))

In [ ]:
for col in str_cleaned.columns:
    dummy = pd.get_dummies(str_cleaned[col], drop_first = True)
    df_norm = pd.concat([df_norm,dummy], axis = 1)

    
num_cleaned.reset_index(inplace = True)
num_cleaned.drop("index", inplace = True, axis = 1)
df_norm = pd.concat([df_norm,num_cleaned["Total Claim Amount"]], axis = 1)
df_norm

In [ ]:
#encoder = OneHotEncoder(handle_unknown='error', drop = "first").fit(data[["gender"]])
#encoded = encoder.transform(data[["gender"]]).toarray()
#encoded

In [ ]:
df_norm

In [ ]:
# Test Split
Y = df_norm["Total Claim Amount"]
X = df_norm.drop(columns = "Total Claim Amount")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, random_state = 100)

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from math import sqrt
lm = linear_model.LinearRegression()
model = lm.fit(X_train, Y_train)
lm.score(X,Y)
print ("lm intercept: ", lm.intercept_,"\n")
print ("lm coeff: ", lm.coef_,"\n")
predictions = lm.predict(X_test)

residuals = predictions - Y_test
sns.distplot(residuals)
plt.show()


mae = mean_absolute_error(Y,predictions)
print ("mae is: ", mae, "\n")
mse = mean_squared_error(Y,predictions)
print ("mse is: ", mse, "\n")
rmse = sqrt(mse)
print ("rmse is: ", rmse, "\n")
r2 = r2_score(Y,predictions)
print ("r2 is: ", r2)

In [ ]:
#### Stats Model
import statsmodels.api as sm
from statsmodels.formula.api import ols
Y = data2["median_home_val"]
X = data2.drop(["median_home_val"], axis = 1)
X = sm.add_constant(X)
model = sm.OLS(Y,X).fit()
print (model.summary())